In [ ]:
from skimage.io import imread
import numpy as np
from scipy.io import loadmat
import scipy as sp
from matplotlib import pyplot as plt
import time

In [ ]:
rootfolder = ".."


Useful function for plot the dictionary


In [ ]:
def get_dictionary_img(D):
    M, N = D.shape
    p = int(round(np.sqrt(M)))
    nnn = int(np.ceil(np.sqrt(N)))
    bound = 2
    img = np.ones((nnn * p + bound * (nnn - 1), nnn * p + bound * (nnn - 1)))
    for i in range(N):
        m = np.mod(i, nnn)
        n = int((i - m) / nnn)
        m = m * p + bound * m
        n = n * p + bound * n
        atom = D[:, i].reshape((p, p))
        if atom.min() < atom.max():
            atom = (atom - atom.min()) / (atom.max() - atom.min())
        img[m : m + p, n : n + p] = atom

    return img

Set all the parameters for the anomaly detection


In [ ]:
# patch size (tha patch is square)
p = 15

# number of patches in the training set for dictionary learning
npatch_dictionary = 10000

# number of patches to estimate the confidence region
npatch_region = 1000

# parameters for the dictionary learning using the KSVD
niter_dl = 10
natom = int(np.round(p**2 * 1.5))
L = 4

# regularization parameters for the l1 sparse coding
lmbda = 0.18

## Construct the training and validation sets


In [ ]:
# load the training image and rescale it in [0,1]
img = imread(f"{rootfolder}/data/img_normal.png") / 255

# extract random patches from the image and store them in a matrices S, V
# S =
# V =

## Dictionary Learning

Perform preprocessing on the patches in $S$


In [ ]:
# PREPROCESSING: exclude black patches from S
v = np.median(S, axis=0)
S = S[:, v > 0.06]

In [ ]:
# PREPROCESSING: remove the mean from each patch
# S =


Perform dictionary learning via KSVD or MOD


In [ ]:
# D =

# or load a precomputed dictionary
# D = loadmat(f'{rootfolder}/data/dict_anom_det.mat')['D']

Show the learned dictionary


In [ ]:
img_dict = get_dictionary_img(D)
plt.imshow(img_dict, cmap="gray")
plt.show()

## Confidence region estimation / density estimation


In [ ]:
# PREPROCESSING: exclude black patches
v = np.median(V, axis=0)
V = V[:, v > 0.06]

In [ ]:
# PREPROCESSING: remove the mean from each patch
V =

In [ ]:
# sparse coding of each patch in V
X = np.zeros((natom, npatch_region))
for i in range(V.shape[1]):
#    X[:, i] =



In [ ]:
# computing the anomaly indicators (l1 norm, reconstruction error) for each
# patch in V

A = np.zeros((2, V.shape[1]))     # each column contains the values of the anomaly_scores for a patch

for i in range(V.shape[1]):
#    A[:,i] =

# Estimation of mean and covariance
#mu =
#Sigma =


In [ ]:
# estimation of the threshold that gives the desired false positive rate
# using the patches in V

FPR_target = 0.1

# compute the Mahalanobis distance for each indicator vector in A
mahal_dist = np.zeros(V.shape[1])
for i in range(A.shape[1]):
#    mahal_dist[i] =

# set the threshold
#threshold =


## Test phase


In [ ]:
# load the test image
img_test = imread(f"{rootfolder}/data/img_anom.png") / 255

imsz = img_test.shape

STEP = 7
# initialize the estimated image
heatmap = np.zeros_like(img)

# initialize the weight matrix
weights = np.zeros_like(img)

In [ ]:
for i in range(0, imsz[0] - p + 1, STEP):
    for j in range(0, imsz[1] - p + 1, STEP):
        # extrach the patch with the top left corner at pixel (ii, jj)
        #s =

        # if the median of s is to small set the anomaly score to 0:
        if np.median(s) <= 0.06:
            score = 0
        else:
            # subratct the mean to the patch
            #s =

            # perform the sparse coding
            #x =

            # compute the anomaly indicators vector
            #a =

            # compute the anomaly score
            #score =

        # update the heatmap
        #heatmap

        # update the weight matrix
        #weights


In [ ]:
# normalize the heatmap
# heatmap =

In [ ]:
# plot the heatmap
plt.imshow(heatmap)

In [ ]:
# build the detection mask, that has the same size of the test image
# each pixel in the mask has value 1 if the corresponding patch has been
# detected as anomalous, otherwise it has value 0
# mask =


In [ ]:
## show the results
plt.figure(3), plt.imshow(img_test, cmap="gray"), plt.title("Test Image")
plt.figure(4), plt.imshow(mask, cmap="gray"), plt.title("Mask")
plt.show()

In [ ]:
# combine the mask and the test image
img_color = np.zeros([img_test.shape[0], img_test.shape[1], 3])
img_temp = img_test.copy()
img_temp[mask > 0] = 1
img_color[:, :, 0] = img_temp
img_temp = img_test.copy()
img_temp[mask > 0] = 0
img_color[:, :, 1] = img_temp
img_temp = img_test.copy()
img_temp[mask > 0] = 0
img_color[:, :, 2] = img_temp

plt.figure(5), plt.imshow(img_color), plt.title("Detections")
plt.show()